In [7]:
# Importar librerias


import pandas as pd
from windrose import WindroseAxes
import itertools
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from math import pi
from windrose import WindroseAxes
import streamlit as st

ModuleNotFoundError: No module named 'streamlit'

In [5]:
# --- Muestra de DataFrame ---

data = {
    'wellName': ['CARDALES_1N','CARDALES_1N', 'CARDALES_1N'],
    'TDEP': [5597, 5607, 5622],     # Profundidad
    'Azimuth': [278, 265, 270],     # Rumbo
    'Dip_RV2025_TRU': [10, 16, 14],     # Buzamiento
    'Type': ['Cross bedding', 'Bedding', 'Partially Open Fracture'],
}
df = pd.DataFrame(data)
print(df)

      wellName  TDEP  Azimuth  Dip_RV2025_TRU                     Type
0  CARDALES_1N  5597      278              10            Cross bedding
1  CARDALES_1N  5607      265              16                  Bedding
2  CARDALES_1N  5622      270              14  Partially Open Fracture


In [6]:
# --- Cargar datos ---
RUTA_CSV = '/workspaces/ml-bootcamp-labs/data/raw/DATOS_DIAGRAMA_ROSA_UNIFICADO.csv'
df = pd.read_csv(RUTA_CSV, sep=';')

st.title('Generador de Diagramas de Rosa')

NameError: name 'st' is not defined